# Webscrapeo : Ubica tu casilla

### Paquetes

In [91]:
import time
import requests
import numpy as np
import pandas as pd

# Local imports
from keys import proxies

requests.adapters.DEFAULT_RETRIES = 5 # increase retries number

### Pipeline

In [92]:
API = "https://us-central1-ine-ubica-tu-casilla.cloudfunctions.net/utc-api/search/rasgo-with-cmr-uts?e=1&s=488"

headers = {"authority": "us-central1-ine-ubica-tu-casilla.cloudfunctions.net",
            "method": "GET",
            "scheme": "https",
            "accept": "application/json, text/plain, */*",
            "accept-encoding": "gzip, deflate, br",
            "origin": "https://ubicatucasilla.ine.mx",
            "referer": "https://ubicatucasilla.ine.mx/",
            "accept-language": "en,es-419;q=0.9,es;q=0.8",
            "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36",
            "x-utc-service-id": "ubicatucasilla",
            "x-utc-version-id": "2022",
            "sec-ch-ua": 'Not A;Brand";v="99", "Chromium";v="100", "Google Chrome";v="100"',
            "sec-ch-ua-mobile": "?0",
            "sec-ch-ua-platform": "Windows",
            "sec-fetch-dest": "empty",
            "sec-fetch-mode": "cors",
            "sec-fetch-site": "cross-site",
}

In [93]:
def get_raw_df(API, num_edo, num_casilla, session):
    """
    Extrae el dataframe de una casilla a partir del API
     
    Parameters
    API: string
        URL del API
    num_edo: int
        Numero de estado
    num_casilla: int
        Numero de casilla a extraer
    """
    path = "/utc-api/search/rasgo-with-cmr-uts?e={num_edo}&s={num_casilla}"
    headers["path"] = path 

    new_API = API.replace("e=1", f"e={num_edo}")
    new_API = new_API.replace("s=488", f"s={num_casilla}")
    success = 0
    while success == 0:
        try:
            r = session.get(new_API, headers=headers, timeout=5)
            success = 1
        except Exception as err:
            time.sleep(1.5)
            print(f"Retrying request: {err}")

    if r.json()["status"] == False:
        raw_df = pd.DataFrame([])
        if 'cmr' in r.json().keys():
            content = r.json()

        else:
            raw_df = pd.DataFrame([])
            content = {}
    
    else:
        content = r.json()['data'][0]['items']
        raw_df = pd.DataFrame(content[0]['apellidos'])
        raw_df['domicilio'] = content[0]['data']['domicilio']
        raw_df['ubicacion'] = content[0]['data']['ubicacion']

        raw_df['entidad'] = content[0]['entidadSede']
        raw_df['distitoSede'] = content[0]['distritoSede']
        raw_df['distritoLocal'] = content[0]['distrito_localSede']
        raw_df['latitud'] = content[0]['reference']['dce']['geom']['coordinates'][0]
        raw_df['longitud'] = content[0]['reference']['dce']['geom']['coordinates'][1]
        
        raw_df 

    return raw_df, content

In [94]:
def get_clean_df(raw_df, content):
    """
    Saca clean_df a partir del raw_df y pone varios 
    dfs de acuerdo al numero de secciones
    
    Parameters
    ----------
    raw_df: dataframe
        Dataframe de la casilla
    content: dict
        Diccionario con la informacion de la casilla
    
    """
    raw_secciones = content[0]['secciones']

    clean_secciones = [int(x) for x in raw_secciones]
    complete_df = pd.DataFrame([])

    for clean_seccion in clean_secciones:
        copy_df = raw_df.copy()
        copy_df['seccion'] = clean_seccion

        complete_df = pd.concat([complete_df, copy_df], 
                                ignore_index=True)

    return complete_df, clean_secciones

In [95]:
def extract_estado(API, num_edo, session):
    """
    Extrae todas las casillas de un estado

    Parameters
    ----------
    API : string
        API de INE
    num_edo : int
        Numero de estado a extraer

    Returns
    -------
    estado_df : dataframe
        Dataframe con todas las casillas de un estado
    pending_sections : list
        Lista con todas las secciones pendientes
    """

    all_secciones = np.arange(1, 7000)
    extracted_sections = []
    estado_df = pd.DataFrame([])
    pending_sections = []
    counter = 0
    non_existent_counter = 0

    for seccion in all_secciones:
        counter += 1
        if counter % 100 == 0:
            print(f"Extracted {counter} casillas")
            time.sleep(20)

        # Si la seccion ya la extrajimos
        if seccion in extracted_sections:
            continue

        raw_df, content = get_raw_df(API, num_edo, seccion, session)

        if content:

            if not isinstance(content, list):
                time.sleep(3)
                pending_sections.append(seccion)
                # Si falla el API, continuamos (si existe la casilla, pero pide datos del INE)
                continue
            else:
                # Sacar datos limpios y quitar casillas que ya hayamos extraido
                complete_df, clean_secciones = get_clean_df(raw_df, content)
                estado_df = pd.concat([estado_df, complete_df], ignore_index=True)
                # Agregar las secciones ya extraidas
                extracted_sections.extend(clean_secciones)

        # Si el content está vacío, continuamos
        else:
            non_existent_counter += 1
            
            if non_existent_counter >= 30:
                print("40 casillas no encontradas, stopping...")
                break
            else:
                continue
    
    return estado_df, pending_sections

In [121]:
def get_casillas_especiales():
    """
    Extrae casillas especiales de la API

    """
    API_especial = "https://us-central1-ine-ubica-tu-casilla.cloudfunctions.net/utc-api/search/rasgo?e=31&special=true"
    specials_df = pd.DataFrame([])

    with requests.Session() as session:
        for estado in np.arange(1, 33):

            new_API = API_especial.replace("e=31", f"e={estado}")
            r = session.get(new_API, headers=headers)
            content = r.json()['data']

            estado_df = pd.DataFrame([])
            for sub_content in content:
                raw_df = pd.DataFrame(sub_content['rasgo']).reset_index()
                raw_df = raw_df.iloc[1:,]
                raw_df.reset_index(drop=True, inplace=True)

                estado_df = pd.concat([estado_df, raw_df], ignore_index=True)
            
            estado_df['entidad'] = estado

            specials_df = pd.concat([specials_df, estado_df], 
                                    ignore_index=True)
            
            time.sleep(1)
        
        requests.post(new_API, headers={"Connection": "close"})

    return specials_df

### Pipeline

In [66]:
estados = np.arange(1, 32)
mexico_df = pd.DataFrame([])
pendientes_dic = {}

session = requests.Session()

for estado in estados:    
    estado_df, pending_sections = extract_estado(API, estado, session)
    pendientes_dic[estado] = pending_sections

    mexico_df = pd.concat([mexico_df, estado_df], ignore_index=True)

    print(f"========== Extracted {estado} ===========")
    time.sleep(20)



Retrying request: HTTPSConnectionPool(host='us-central1-ine-ubica-tu-casilla.cloudfunctions.net', port=443): Max retries exceeded with url: /utc-api/search/rasgo-with-cmr-uts?e=20&s=1 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001B964C34D30>: Failed to establish a new connection: [WinError 10048] Only one usage of each socket address (protocol/network address/port) is normally permitted'))
Retrying request: HTTPSConnectionPool(host='us-central1-ine-ubica-tu-casilla.cloudfunctions.net', port=443): Max retries exceeded with url: /utc-api/search/rasgo-with-cmr-uts?e=20&s=1 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001B964C8E9D0>: Failed to establish a new connection: [WinError 10048] Only one usage of each socket address (protocol/network address/port) is normally permitted'))
Retrying request: HTTPSConnectionPool(host='us-central1-ine-ubica-tu-casilla.cloudfunctions.net', port=443): Max retries exceeded with u

KeyboardInterrupt: 

In [ ]:
mexico_df

,casilla,inicial,final,domicilio,ubicacion,entidad,distitoSede,distritoLocal,seccion
0,B,ABOYTES DOÑAS,LLAMAS DELGADO,"CALLE HUIZACHE, NÚMERO 108, FRACCIONAMIENTO EL...",DOMICILIO PARTICULAR,1,3,6,1
1,C1,LLAMAS ESPERON,ZUÑIGA GONZALEZ,"CALLE HUIZACHE, NÚMERO 108, FRACCIONAMIENTO EL...",DOMICILIO PARTICULAR,1,3,6,1
2,B,ABREGO GUTIERREZ,LLAMAS VAZQUEZ,"AVENIDA GUADALUPE GONZÁLEZ, SIN NÚMERO, COLONI...",AULA MAGNA DEL DESARROLLO INTEGRAL DE LA FAMIL...,1,3,6,2
3,C1,LOERA CHAVEZ,ZURITA ALVARADO,"AVENIDA GUADALUPE GONZÁLEZ, SIN NÚMERO, COLONI...",AULA MAGNA DEL DESARROLLO INTEGRAL DE LA FAMIL...,1,3,6,2
4,B,ABREGO GUTIERREZ,LLAMAS VAZQUEZ,"AVENIDA GUADALUPE GONZÁLEZ, SIN NÚMERO, COLONI...",AULA MAGNA DEL DESARROLLO INTEGRAL DE LA FAMIL...,1,3,6,4
...,...,...,...,...,...,...,...,...,...
99115,C3,RANGEL MALO,ZUÑIGA TORRES,"CALLE CARRILLO, SIN NÚMERO, FRACCIONAMIENTO PR...",ESCUELA PRIMARIA YSIDRO SEPÚLVEDA GONZÁLEZ,19,8,14,647
99116,B,ABREGO HERNANDEZ,ESCOBEDO SALINAS,"CALLE CARRILLO, SIN NÚMERO, FRACCIONAMIENTO PR...",ESCUELA PRIMARIA YSIDRO SEPÚLVEDA GONZÁLEZ,19,8,14,2910
99117,C1,ESCOBEDO SALINAS,LOPEZ PARTIDA,"CALLE CARRILLO, SIN NÚMERO, FRACCIONAMIENTO PR...",ESCUELA PRIMARIA YSIDRO SEPÚLVEDA GONZÁLEZ,19,8,14,2910
99118,C2,LOPEZ PATIÑO,RANGEL LOERA,"CALLE CARRILLO, SIN NÚMERO, FRACCIONAMIENTO PR...",ESCUELA PRIMARIA YSIDRO SEPÚLVEDA GONZÁLEZ,19,8,14,2910


In [ ]:
mexico_df.to_csv("casillas_edos-1-32.csv", encoding="utf-8-sig", 
                    index=False)        

In [248]:
raw_df, content = get_raw_df(API, 1, 98)

In [78]:
new_API = API.replace("e=1", f"e={20}")
new_API = new_API.replace("s=488", f"s={6}")
r = requests.get(new_API, headers=headers, proxies=proxies)
r.json()['data'][0]['items'][0]['reference']['dce']['geom']['coordinates']

[-96.457847345, 18.507701639]

In [118]:
specials_df

,index,id,active,type,domicilio,ubicacion,referencia,id_domicilio,croquis,geom,dce,name,alias,entidad
0,coordinates,030101260100,True,3,"AVENIDA AGUASCALIENTES ORIENTE, SIN NÚMERO, CO...",ESTACIÓN DE BOMBEROS,ENTRE PUENTE DE ALVARADO Y AVENIDA TECNOLÓGICO,79234,True,"[-102.256193237, 21.875532845]","[-102.256193319, 21.875532845]",S1,Especial 1,1
1,coordinates,030102180100,True,3,"AVENIDA DE LA CONVENCIÓN DE 1914 SUR, NÚMERO 4...",CENTRAL DE ÓMNIBUS EN AGUASCALIENTES,ENTRE CALLE QUINTA AVENIDA Y CALLE REPÚBLICA D...,77108,True,"[-102.298926129, 21.865877385]","[-102.298952933, 21.865887342]",S1,Especial 1,1
2,coordinates,030104700100,True,3,"AVENIDA JUAREZ, NÚMERO 412, COLONIA CENTRO,203...",JARDIN DE NIÑOS MARÍA LUISA VILLA DE GARCÍA ROJAS,RETORNO CARRETERA AGSZAC Y JOSE MARIA MORELOS,79045,True,"[-102.272574758, 22.083958521]","[-102.272511724, 22.083954793]",S1,Especial 1,1
3,coordinates,030202090100,True,3,"CARRETERA TRANSPENINSULAR, NÚMERO 6500, EX EJI...",CENTRO DE GOBIERNO,A UN COSTADO DE PALACIO MUNICIPAL,73492,True,"[-116.59466588, 31.808960466]","[-116.594665888, 31.808978701]",S1,Especial 1,2
4,coordinates,030202460100,True,3,"CALZADA ADOLFO LÓPEZ MATEOS, SIN NÚMERO, CONJU...",TEATRO DEL ESTADO,ESQUINA CON CALLE MILTON CASTELLANOS,974,True,"[-115.474754, 32.65468]","[-115.476432432, 32.65508607]",S1,Especial 1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,coordinates,033109040100,True,3,"CALLE 15, SIN NÚMERO, PISTÉ, CÓDIGO POSTAL 977...",COLEGIO DE BACHILLERES DEL ESTADO DE YUCATÁN P...,CERCA DEL HOTEL PIRÁMIDE INN RESORT,82405,True,"[-88.581873241, 20.693077743]","[-88.581873173, 20.693075288]",S1,Especial 1,31
296,coordinates,033201890100,True,3,"CALLE DURANGO, NÚMERO 402, COLONIA CENTRO, CÓD...",INTERIOR DE LA CENTRAL DE AUTOBUSES,ENTRE CALLE ÉBANO Y PASEO DEL MINERAL,70278,True,"[-102.859379, 23.173519]","[-102.859376372, 23.17351679]",S1,Especial 1,32
297,coordinates,033204700100,True,3,"CALZADA REVOLUCIÓN MEXICANA SIN NÚMERO, COLONI...",CENTRO DE CAPACITACIÓN PARA EL TRABAJO INDUSTR...,"ENTRONQUE A SAUCEDA DE LA BORDA COLONIA LA FE,...",72942,True,"[-102.509095832, 22.756358478]","[-102.509090425, 22.756360951]",S1,Especial 1,32
298,coordinates,033206140100,True,3,"CALLE PUEBLA ESQUINA CALLE CENTRAL, SIN NÚMERO...",ESCUELA PRIMARIA FRANCISCO GOITIA,ENTRE CALLE DEL ROSARIO Y CALLE CENTRAL,None,False,"[-102.985023134, 22.65423736]","[-102.985109346, 22.654728965]",S1,Especial 1,32


In [ ]:
specials_df.to_csv("casillas_especiales.csv", encoding="utf-8-sig",
                    index=False)